In [1]:
from pennylane import numpy as np
import pennylane as qml
import openfermion

In [2]:
symbors = ["N", "N"]
dist = 2.0741632451/2
coordinates = np.array([0.0, 0.0, -dist, 0.0, 0.0, dist ])

In [3]:
H, qubits = qml.qchem.molecular_hamiltonian(symbors, coordinates)
print("Number of qubits = ", qubits)
print("The Hamiltonian is ", H)

Number of qubits =  20
The Hamiltonian is    (-66.1758138466603) [I0]
+ (1.0077010976727858) [Z18]
+ (1.0077010976727858) [Z19]
+ (1.1675851256971157) [Z14]
+ (1.1675851256971157) [Z15]
+ (1.1675851256972902) [Z16]
+ (1.1675851256972907) [Z17]
+ (1.2845149023621105) [Z12]
+ (1.2845149023621114) [Z13]
+ (1.337444418458456) [Z7]
+ (1.3374444184584564) [Z6]
+ (1.3454212925604765) [Z10]
+ (1.3454212925604767) [Z11]
+ (1.3454212925605864) [Z8]
+ (1.3454212925605864) [Z9]
+ (1.8386664428761654) [Z4]
+ (1.8386664428761663) [Z5]
+ (9.609117365598106) [Z2]
+ (9.609117365598108) [Z3]
+ (9.609253513874302) [Z1]
+ (9.609253513874304) [Z0]
+ (-7.79659655694239e-10) [Y7 Y9]
+ (-7.79659655694239e-10) [X7 X9]
+ (-1.2105449779628073e-10) [Y0 Y2]
+ (-1.2105449779628073e-10) [X0 X2]
+ (1.9579317655552512e-11) [Y2 Y4]
+ (1.9579317655552512e-11) [X2 X4]
+ (1.2107262794502642e-10) [Y1 Y3]
+ (1.2107262794502642e-10) [X1 X3]
+ (9.152180105350145e-10) [Y6 Y8]
+ (9.152180105350145e-10) [X6 X8]
+ (0.083377353347

In [4]:
dev = qml.device("default.qubit", wires=qubits)

In [5]:
electrons = 10
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]


In [18]:
def circuit(param, wires):
    qml.BasisState(hf, wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 10, 11,] )

In [19]:
@qml.qnode(dev, interface="autograd")
def cost_fn(param):
    circuit(param, wires=range(qubits))
    return qml.expval(H)

In [20]:
opt = qml.GradientDescentOptimizer(stepsize=0.4)

In [21]:
theta = np.array(0.0, requires_grad=True)

In [22]:
energy = [cost_fn(theta)]

# store the values of the circuit parameter
angle = [theta]

max_iterations = 100
conv_tol = 1e-06

for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_fn, theta)

    energy.append(cost_fn(theta))
    angle.append(theta)

    conv = np.abs(energy[-1] - prev_energy)

    if n % 2 == 0:
        print(f"Step = {n},  Energy = {energy[-1]:.8f} Ha")

    if conv <= conv_tol:
        break

print("\n" f"Final value of the ground-state energy = {energy[-1]:.8f} Ha")
print("\n" f"Optimal value of the circuit parameter = {angle[-1]:.4f}")